In [11]:
import os
import librosa
import numpy as np
import pandas as pd

# 🔹 소음 카테고리별 원천 데이터(WAV) 폴더
noise_categories = {
    "차량경적": "F:/noise_project/원천데이터/1.자동차/1.차량경적",
    "차량사이렌": "F:/noise_project/원천데이터/1.자동차/2.차량사이렌",
    "차량주행음": "F:/noise_project/원천데이터/1.자동차/3.차량주행음",
    "이륜차경적": "F:/noise_project/원천데이터/2.이륜자동차/4.이륜차경적",
    "이륜차주행음": "F:/noise_project/원천데이터/2.이륜자동차/5.이륜차주행음"
}

# 🔹 분석 결과 저장용 리스트
mono_files = []
stereo_files = []
direction_results = []

# 🔹 좌/우 방향성 판단 기준 (데시벨 차이)
DIRECTION_THRESHOLD = 0.001  # 데시벨 차이가 1.5 이상이면 방향 판별

# 🔹 각 소음 카테고리별 분석
for category, audio_folder in noise_categories.items():
    print(f"\n🔎 {category} 소음 분석 시작...")

    # 🔹 원천 데이터(WAV) 파일 리스트 가져오기
    wav_files = [f for f in os.listdir(audio_folder) if f.endswith(".wav")]
    total_wav_files = len(wav_files)

    for wav_file in wav_files:
        try:
            audio_path = os.path.join(audio_folder, wav_file)
            y, sr = librosa.load(audio_path, sr=None, mono=False)

            # 🔹 모노 / 스테레오 구분
            if len(y.shape) == 1:  # 모노 (단일 채널)
                mono_files.append((wav_file, category))
                continue  # 모노 파일은 방향 분석 X

            else:  # 스테레오 (좌/우 채널)
                stereo_files.append((wav_file, category))

                # 좌/우 채널 분리
                left_channel, right_channel = y

                # RMS 데시벨 계산
                left_rms = np.sqrt(np.mean(left_channel ** 2))
                right_rms = np.sqrt(np.mean(right_channel ** 2))

                left_db = 20 * np.log10(left_rms + 1e-6)
                right_db = 20 * np.log10(right_rms + 1e-6)

                # 좌/우 방향 판단
                if abs(left_db - right_db) < DIRECTION_THRESHOLD:
                    direction = "중앙"
                elif left_db > right_db:
                    direction = "왼쪽"
                else:
                    direction = "오른쪽"

                direction_results.append((wav_file, category, left_db, right_db, direction))

        except Exception as e:
            print(f"❌ 분석 불가 파일: {wav_file}, 오류: {e}")

    # 🔹 결과 출력 (VSCode 터미널)
    print(f"📂 총 WAV 파일 개수: {total_wav_files}")
    print(f"🎙️ 모노 파일 개수: {len(mono_files)}")
    print(f"🎧 스테레오 파일 개수: {len(stereo_files)}")
    print(f"📊 방향 판단 결과 (상위 10개): {direction_results[:10]}")
    print("-" * 50)

# 🔹 분석 결과 CSV 파일로 저장
output_folder = "F:/noise_project/분석결과"
os.makedirs(output_folder, exist_ok=True)  # 결과 저장 폴더 생성

# ✅ 모노 파일 목록 저장
pd.DataFrame(mono_files, columns=["파일명", "소음 종류"]).to_csv(f"{output_folder}/mono_files.csv", index=False)

# ✅ 스테레오 파일 목록 저장
pd.DataFrame(stereo_files, columns=["파일명", "소음 종류"]).to_csv(f"{output_folder}/stereo_files.csv", index=False)

# ✅ 방향 판단 결과 저장
pd.DataFrame(direction_results, columns=["파일명", "소음 종류", "좌측 dB", "우측 dB", "예측 방향"]).to_csv(f"{output_folder}/direction_results.csv", index=False)

print("✅ 분석 결과 CSV 저장 완료!")




🔎 차량경적 소음 분석 시작...
📂 총 WAV 파일 개수: 3189
🎙️ 모노 파일 개수: 3092
🎧 스테레오 파일 개수: 97
📊 방향 판단 결과 (상위 10개): [('1.자동차_102_1.wav', '차량경적', -9.14025907072723, -11.791679953967027, '왼쪽'), ('1.자동차_106_1.wav', '차량경적', -21.117919183554804, -22.385535344981406, '왼쪽'), ('1.자동차_108_1.wav', '차량경적', -19.650180561979756, -19.701206528802423, '왼쪽'), ('1.자동차_110_1.wav', '차량경적', -12.814280044699373, -14.402563426567056, '왼쪽'), ('1.자동차_111_1.wav', '차량경적', -19.77418251187723, -20.92838135983104, '왼쪽'), ('1.자동차_112_1.wav', '차량경적', -20.4798700407332, -19.655999476235586, '오른쪽'), ('1.자동차_114_1.wav', '차량경적', -19.046822938736657, -18.58490752092631, '오른쪽'), ('1.자동차_116_1.wav', '차량경적', -13.260196993569501, -12.503355382798201, '오른쪽'), ('1.자동차_127_1.wav', '차량경적', -10.473343449860018, -16.14269646250396, '왼쪽'), ('1.자동차_128_1.wav', '차량경적', -30.75666815570408, -29.713842474094285, '오른쪽')]
--------------------------------------------------

🔎 차량사이렌 소음 분석 시작...
📂 총 WAV 파일 개수: 1990
🎙️ 모노 파일 개수: 4909
🎧 스테레오 파일 개수: 270
📊 방향 판단 결과

In [12]:


import pandas as pd

# CSV 파일 경로
csv_path = "F:/noise_project/분석결과/direction_results.csv"

# CSV 파일 불러오기
df = pd.read_csv(csv_path)

# 방향별 비율 계산
direction_counts = df["예측 방향"].value_counts(normalize=True) * 100

# 결과 출력
direction_counts


예측 방향
오른쪽    49.903784
왼쪽     41.180244
중앙      8.915972
Name: proportion, dtype: float64